In [137]:
import pandas as pd
import json
import collections
import scipy.io
import numpy as np

In [189]:
bus = scipy.io.loadmat('MATPOWER/bus.mat')['bus']
gen = scipy.io.loadmat('MATPOWER/gen.mat')['gen']
power_demand = bus[:, 2]
df = pd.read_csv("GIS/CATS_gens.csv")
#df = df.drop_duplicates(subset=['Lat','Lon'])

In [190]:
generator = gen[:,0][gen[:,1] != 0.0]

In [179]:
res = df[df['Pg'] == 0.0]

In [191]:
len(generator)

1800

In [181]:
res.head()

,PlantCode,GenID,bus,FuelType,Pg,Pmax,Pmin,Qg,Qmax,Qmin,Lat,Lon
1,151,1,1804,Petroleum Liquids,0.0,71.2,0,0,34.5,-23.7,37.62936,-120.93139
2,151,2,1804,Natural Gas Fired Combustion Turbine,0.0,71.2,0,0,34.5,-23.7,37.62936,-120.93139
106,315,3,8113,Natural Gas Steam Turbine,0.0,310.0,0,0,0.0,0.0,33.76880,-118.10090
107,315,4,8113,Natural Gas Steam Turbine,0.0,310.0,0,0,0.0,0.0,33.76880,-118.10090
108,315,5,8113,Natural Gas Steam Turbine,0.0,495.0,0,0,0.0,0.0,33.76880,-118.10090


In [180]:
len(res['PlantCode'])

2092

In [159]:

gen_fuel_type = df['FuelType'].to_numpy()

1016

In [141]:
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
f = open("GIS/CATS_lines.json")
CATS_lines = json.load(f)
i = 0
voltage_levels = []
line_to_nodes = []
for feature in CATS_lines['features']:
    voltage_levels.append(feature['properties']['kV'])
    line_to_nodes.append((feature['properties']['f_bus'], feature['properties']['t_bus']))
    graph[feature['properties']['f_bus']].append((feature['properties']['t_bus'], i))
    graph_reverse[feature['properties']['t_bus']].append((feature['properties']['f_bus'], i))
    i += 1
f.close()

In [187]:
f = open("pf_solution.json")
sol = json.load(f)
gen = []
bus = []
branch = []
gen_cost = []
power_generation = []
branch_power_to = []
branch_power_from = []
for line, val in sol['solution']['gen'].items():
    if val['pg'] != 0.0:
        gen.append(int(line))
        gen_cost.append(val['pg_cost'])
        power_generation.append(val['pg'])
        print(val)
for line, val in sol['solution']['branch'].items():
    branch.append(int(line))
    branch_power_from.append(val['pf'])
    branch_power_to.append(val['pt'])
for line, val in sol['solution']['bus'].items():
    bus.append(int(line))
f.close()

{'qg': None, 'pg': 0.12929218391099856, 'pg_cost': 0.0}
{'qg': None, 'pg': -8.777240656023013e-09, 'pg_cost': 401.19410563015464}
{'qg': None, 'pg': 0.1358427574422526, 'pg_cost': 687.9341526476228}
{'qg': None, 'pg': -9.609564847572118e-09, 'pg_cost': 632.8730379998229}
{'qg': None, 'pg': 0.387876531912902, 'pg_cost': 0.0}
{'qg': None, 'pg': 0.0064646186161296795, 'pg_cost': 0.0}
{'qg': None, 'pg': 0.025858444713830885, 'pg_cost': 0.0}
{'qg': None, 'pg': 0.7165858228458662, 'pg_cost': 2163.10267251983}
{'qg': None, 'pg': 0.2672268632539292, 'pg_cost': 710.290107915472}
{'qg': None, 'pg': 0.01939383601434581, 'pg_cost': 0.0}
{'qg': None, 'pg': 0.07000000864793814, 'pg_cost': 144.33561961824512}
{'qg': None, 'pg': -9.96466835004798e-09, 'pg_cost': 173.52994130112842}
{'qg': None, 'pg': -9.96352380712013e-09, 'pg_cost': 173.52994130787056}
{'qg': None, 'pg': 0.012000009854335698, 'pg_cost': 31.652314229311475}
{'qg': None, 'pg': -9.963693026758635e-09, 'pg_cost': 173.52994130687375}
{'qg

In [143]:
num_bus = len(bus)
num_branch = len(branch)
num_gen = len(gen)
print("Number of transmission lines and transformers: " + str(num_branch))
print("Number of buses: " + str(num_bus))
print("Number of generators: " + str(num_gen))

Number of transmission lines and transformers: 10823
Number of buses: 8870
Number of generators: 2149


In [128]:
line_to_gen = collections.defaultdict(set)
node_to_gen = collections.defaultdict(set)
def dfs(g, n, visited):
    if n in visited: return
    visited.add(n)
    node_to_gen[n].add(g)
    for nei, line in graph[n]:
        line_to_gen[line].add(g)
        dfs(g, nei, visited)
for i, g in enumerate(gen):
    visited = set()
    dfs(g, g, visited)

In [132]:
line_prop_mat=np.zeros((num_gen, num_branch), dtype=float)
bus_prop_mat=np.zeros((num_gen, num_bus), dtype=float)
# initially, if node_to_gen only has 1 generator, bus_prop_mat start with 1
for k, v in node_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        bus_prop_mat[idx][int(k)] = 1.0
for k, v in line_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        line_prop_mat[idx][int(k)] = 1.0

In [147]:
# visit nodes in topological order
# step 1: calculate bus_prop based on in flowing lines_prop
# step 2: calculate line_prop of out flowing lines
in_degree = collections.defaultdict(int)
for i, v in graph_reverse.items():
    in_degree[i] = len(v)
q = [] # list of nodes with no inflow
topo_order = []
for g in gen:
    if in_degree[g] == 0:
        q.append(g)
while q:
    cur = q.pop(0)
    if len(node_to_gen[cur]) > 1:
        out_total = power_demand[cur]
        for nei, out_line in graph[cur]:
            out_total += branch_power_from[out_line]
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            if cur == g:
                bus_prop_mat[idx][cur] = power_generation[idx]/out_total
            else:
                for nei, in_line in graph_reverse[cur]:
                    bus_prop_mat[idx][cur] += branch_power_from[in_line]*line_prop_mat[idx][in_line]/out_total
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            for nei, out_line in graph[cur]:
                line_prop_mat[idx][out_line] = bus_prop_mat[idx][cur]

    topo_order.append(cur)
    for nei, line in graph[cur]:
        in_degree[nei] -= 1
        if in_degree[nei] == 0:
            q.append(nei)

C:\Users\syq41\AppData\Local\Temp\ipykernel_17780\2586379033.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  bus_prop_mat[idx][cur] = power_generation[idx]/out_total
C:\Users\syq41\AppData\Local\Temp\ipykernel_17780\2586379033.py:24: RuntimeWarning: divide by zero encountered in scalar divide
  bus_prop_mat[idx][cur] += branch_power_from[in_line]*line_prop_mat[idx][in_line]/out_total
C:\Users\syq41\AppData\Local\Temp\ipykernel_17780\2586379033.py:24: RuntimeWarning: invalid value encountered in scalar divide
  bus_prop_mat[idx][cur] += branch_power_from[in_line]*line_prop_mat[idx][in_line]/out_total
C:\Users\syq41\AppData\Local\Temp\ipykernel_17780\2586379033.py:24: RuntimeWarning: invalid value encountered in scalar add
  bus_prop_mat[idx][cur] += branch_power_from[in_line]*line_prop_mat[idx][in_line]/out_total
